In [61]:
import pandas as pd
import geopandas
import datetime
from geopandas import GeoDataFrame
from shapely.geometry import Point
import matplotlib.pyplot as plt
from shapely.ops import nearest_points
from math import sin, cos, sqrt, atan2, radians
import math
from numpy import cos, sin, arcsin, sqrt
from IPython.display import display

In [14]:
#import fire calls into pandas
fireCalls = pd.read_csv("Seattle_Real_Time_Fire_911_Calls.csv")

#remove unnecessary initial line
fireCalls = fireCalls.drop([0, 0])

#add columns to break down data further
#https://erikrood.com/Python_References/extract_month_year_pandas_final.html
fireCalls['Month'] = pd.DatetimeIndex(fireCalls['Datetime']).month
fireCalls['Year'] = pd.DatetimeIndex(fireCalls['Datetime']).year

KeyboardInterrupt: 

In [ ]:
#separate 2017 and 2018 record
fc2017 = fireCalls[fireCalls.Year == 2017]
fc2018 = fireCalls[fireCalls.Year == 2018]

In [9]:
#import seattle neighborhood shapefile
seaNH = geopandas.read_file("City_Clerk_Neighborhoods\City_Clerk_Neighborhoods.shp")

#need to remove excess rows at beginning for extra islands
seaNH = seaNH.drop(seaNH.index[:26])

#get seaNH projection data to use for fire calls geodata frame 
seaNH_crs = seaNH.crs

In [ ]:
#convert fireCalls to geodataframe
#https://gis.stackexchange.com/questions/174159/convert-a-pandas-dataframe-to-a-geodataframe
geometry1 = [Point(xy) for xy in zip(fc2017.Longitude, fc2017.Latitude)]
crs = seaNH_crs
fc2017_geo = GeoDataFrame(fc2017, crs = crs, geometry = geometry1)

geometry2 = [Point(xy) for xy in zip(fc2018.Longitude, fc2018.Latitude)]
crs = seaNH_crs
fc2018_geo = GeoDataFrame(fc2018, crs = crs, geometry = geometry2)


In [ ]:
#spatial join fc_geo to neighborhoods so each fire call has a neighborhood attribute
fc2017_seattle = geopandas.sjoin(fc2017_geo, seaNH, how = 'inner', op = 'within')
fc2018_seattle = geopandas.sjoin(fc2018_geo, seaNH, how = 'inner', op = 'within')

In [58]:
fc2017_seattle.to_file("2017_Fire_Calls_Seattle")
fc2018_seattle.to_file("2018_Fire_Calls_Seattle")

NameError: name 'fc2017_seattle' is not defined

In [10]:
fc2017 = geopandas.read_file("2017_Fire_Calls_Seattle/2017_Fire_Calls_Seattle.shp")
fc2018 = geopandas.read_file("2018_Fire_Calls_Seattle/2018_Fire_Calls_Seattle.shp")

In [11]:
fireS = pd.read_csv("Fire_Stations.csv")
hospital = pd.read_csv("Hospitals.csv")

In [12]:
fs_geom = [Point(xy) for xy in zip(fireS.X, fireS.Y)]
crs = seaNH_crs
fs_geo = GeoDataFrame(fireS, crs = crs, geometry = fs_geom)

In [13]:
hos_geom = [Point(xy) for xy in zip(hospital.X, hospital.Y)]
crs = seaNH_crs
hospital_geo = GeoDataFrame(hospital, crs = crs, geometry = hos_geom)

In [14]:
pts3 = fs_geo.geometry.unary_union
def near(point, pts=pts3):
    nearest = fs_geo.geometry == nearest_points(point, pts)[1]
    return fs_geo[nearest].STNID.get_values()[0]
fc2017['Nearest_Fire_Station'] = fc2017.apply(lambda row: near(row.geometry), axis=1)
fc2018['Nearest_Fire_Station'] = fc2018.apply(lambda row: near(row.geometry), axis=1)

In [15]:
pts3 = hospital_geo.geometry.unary_union
def near2(point, pts=pts3):
    nearest = hospital_geo.geometry == nearest_points(point, pts)[1]
    return hospital_geo[nearest].OBJECTID.get_values()[0]
fc2017['Nearest_Hospital'] = fc2017.apply(lambda row: near2(row.geometry), axis=1)
fc2018['Nearest_Hosptial'] = fc2018.apply(lambda row: near2(row.geometry), axis=1)

In [16]:
fc17test = fc2017

In [17]:
fc18test = fc2018

In [19]:
fireS = fireS.drop(columns=['AREA', 'PERIMETER', 'FIRE_', 'FIRE_ID', 'B1', 'STREETS_', 'STREETS_ID', 'STREETS_SI', 'CIP', 'SE_ANNO_CAD_DATA', 'NAME1'])
hospital = hospital.drop(columns=['SE_ANNO_CAD_DATA', 'ACUTE_CARE', 'GIS_EDT_DT'])

In [20]:
fc17test = fc17test.rename(index=str, columns={"Nearest_Fire_Station": "STNID", "Nearest_Hospital": "HOSID"})
hospital = hospital.rename(index=str, columns={"OBJECTID": "HOSID"})

In [21]:
fc17test = fc17test.merge(fireS, on="STNID")
fc17test = fc17test.merge(hospital, on="HOSID")

In [22]:
fc18test = fc18test.rename(index=str, columns={"Nearest_Fire_Station": "STNID", "Nearest_Hosptial": "HOSID"})

In [23]:
fc18test = fc18test.merge(fireS, on="STNID")
fc18test = fc18test.merge(hospital, on="HOSID")

In [53]:
def calculate_distance(row):
    lon1 = row['Longitude']
    lat1 = row['Latitude']
    lon2 = row['X_x']
    lat2 = row['Y_x']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c
    return km

fc17test['FS_Distance'] = fc17test.apply(lambda row: calculate_distance(row), axis=1)

In [54]:
fc18test['FS_Distance'] = fc18test.apply(lambda row: calculate_distance(row), axis=1)

In [122]:
def calculate_distance2(row):
    lon1 = row['Longitude']
    lat1 = row['Latitude']
    lon2 = row['X_y']
    lat2 = row['Y_y']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c
    return km

fc17test['Hos_Distance'] = fc17test.apply(lambda row: calculate_distance2(row), axis=1)
fc18test['Hos_Distance'] = fc18test.apply(lambda row: calculate_distance2(row), axis=1)

In [115]:
fc2017 = fc17test
fc2018 = fc18test

fc2017.drop(columns=['geometry_x', 'geometry_y'])
fc2018.drop(columns=['geometry_x', 'geometry_y'])



,Address,Type,Datetime,Latitude,Longitude,Report Loc,Incident N,Month,Year,index_righ,OBJECTID_x,PERIMETER,S_HOOD,L_HOOD,L_HOODID,SYMBOL,SYMBOL2,AREA,HOODS_,HOODS_ID,SHAPE_Leng,SHAPE_Area,STNID,HOSID,X_x,Y_x,OBJECTID_y,ADDRESS_x,X_y,Y_y,FACILITY,ADDRESS_y,CITY,FS_Distance,Hos_Distance
0,336 25th Av E,Aid Response,01/01/2018 12:53:00 AM +0000,47.621907,-122.299869,"(47.621907, -122.299869)",F180000028,1.0,2018.0,100,101,25522.346,Stevens,CAPITOL HILL,11,3,23,26039592.0,103,149,25522.345637,2.603959e+07,SFD 34,17,-122.291334,47.626000,14,633 32ND AV E,-122.312276,47.620136,Group Health Central Hospital,201 16th Ave E,"Seattle, WA, 98112-5226",0.784558,0.949886
1,616 29th Av E,Aid Response Yellow,01/02/2018 08:50:00 AM +0000,47.624549,-122.295529,"(47.624549, -122.295529)",F180000502,1.0,2018.0,100,101,25522.346,Stevens,CAPITOL HILL,11,3,23,26039592.0,103,149,25522.345637,2.603959e+07,SFD 34,17,-122.291334,47.626000,14,633 32ND AV E,-122.312276,47.620136,Group Health Central Hospital,201 16th Ave E,"Seattle, WA, 98112-5226",0.353149,1.346778
2,23rd Av E / E Aloha St,Aid Response,01/01/2018 08:16:00 PM +0000,47.626759,-122.302382,"(47.626759, -122.302382)",F180000358,1.0,2018.0,100,101,25522.346,Stevens,CAPITOL HILL,11,3,23,26039592.0,103,149,25522.345637,2.603959e+07,SFD 34,17,-122.291334,47.626000,14,633 32ND AV E,-122.312276,47.620136,Group Health Central Hospital,201 16th Ave E,"Seattle, WA, 98112-5226",0.831690,1.044414
3,522 28th Av E,Aid Response,01/09/2018 05:46:00 PM +0000,47.623839,-122.296608,"(47.623839, -122.296608)",F180003203,1.0,2018.0,100,101,25522.346,Stevens,CAPITOL HILL,11,3,23,26039592.0,103,149,25522.345637,2.603959e+07,SFD 34,17,-122.291334,47.626000,14,633 32ND AV E,-122.312276,47.620136,Group Health Central Hospital,201 16th Ave E,"Seattle, WA, 98112-5226",0.462277,1.243567
4,E Roy St / 26th Av E,Aid Response,01/16/2018 10:00:00 AM +0000,47.625169,-122.298760,"(47.625169, -122.29876)",F180005565,1.0,2018.0,100,101,25522.346,Stevens,CAPITOL HILL,11,3,23,26039592.0,103,149,25522.345637,2.603959e+07,SFD 34,17,-122.291334,47.626000,14,633 32ND AV E,-122.312276,47.620136,Group Health Central Hospital,201 16th Ave E,"Seattle, WA, 98112-5226",0.563771,1.156546
5,453 27th Av E,Aid Response,01/21/2018 01:23:00 PM +0000,47.623084,-122.297691,"(47.623084, -122.297691)",F180007428,1.0,2018.0,100,101,25522.346,Stevens,CAPITOL HILL,11,3,23,26039592.0,103,149,25522.345637,2.603959e+07,SFD 34,17,-122.291334,47.626000,14,633 32ND AV E,-122.312276,47.620136,Group Health Central Hospital,201 16th Ave E,"Seattle, WA, 98112-5226",0.575935,1.140465
6,24th Av E / E Highland Dr,Aid Response,01/21/2018 08:50:00 AM +0000,47.630271,-122.301648,"(47.630271, -122.301648)",F180007364,1.0,2018.0,100,101,25522.346,Stevens,CAPITOL HILL,11,3,23,26039592.0,103,149,25522.345637,2.603959e+07,SFD 34,17,-122.291334,47.626000,14,633 32ND AV E,-122.312276,47.620136,Group Health Central Hospital,201 16th Ave E,"Seattle, WA, 98112-5226",0.906547,1.379149
7,23rd Av E / E Aloha St,MVI - Motor Vehicle Incident,01/25/2018 01:35:00 PM +0000,47.626759,-122.302382,"(47.626759, -122.302382)",F180009258,1.0,2018.0,100,101,25522.346,Stevens,CAPITOL HILL,11,3,23,26039592.0,103,149,25522.345637,2.603959e+07,SFD 34,17,-122.291334,47.626000,14,633 32ND AV E,-122.312276,47.620136,Group Health Central Hospital,201 16th Ave E,"Seattle, WA, 98112-5226",0.831690,1.044414
8,2617 E Valley St,Aid Response,01/26/2018 12:28:00 AM +0000,47.625913,-122.298547,"(47.625913, -122.298547)",F180009411,1.0,2018.0,100,101,25522.346,Stevens,CAPITOL HILL,11,3,23,26039592.0,103,149,25522.345637,2.603959e+07,SFD 34,17,-122.291334,47.626000,14,633 32ND AV E,-122.312276,47.620136,Group Health Central Hospital,201 16th Ave E,"Seattle, WA, 98112-5226",0.540279,1.212212
9,728 21st Av E,Aid Response,01/30/2018 02:21:00 PM +0000,47.625436,-122.304724,"(47.625436, -122.304724)",F180010969,1.0,2018.0,100,101,25522.346,Stevens,CAPITOL HILL,11,3,23,26039592.0,103,149,25522.345637,2.603959e+07,SF

In [117]:
pd.options.display.max_columns = None
geometry1 = [Point(xy) for xy in zip(fc17test.Longitude, fc17test.Latitude)]
crs = seaNH_crs
fc2017_geo = GeoDataFrame(fc17test, crs = crs, geometry = geometry1)

geometry2 = [Point(xy) for xy in zip(fc18test.Longitude, fc18test.Latitude)]
crs = seaNH_crs
fc2018_geo = GeoDataFrame(fc18test, crs = crs, geometry = geometry2)

In [125]:
fc17test.to_csv()
fc18test.to_csv()

UnicodeEncodeError: 'ascii' codec can't encode character u'\xb0' in position 2: ordinal not in range(128)

In [127]:
fc17test.to_file('blah1')
fc18test.to_file('blah2')

AttributeError: 'DataFrame' object has no attribute 'to_file'